In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle/

In [ ]:
! cp '/content/drive/MyDrive/kaggle/kaggle.json' ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
! unzip /content/chest-xray-pneumonia.zip -d pneumonia_dataset

In [ ]:
import os
# Directory with our training PNEUMONIA pictures
train_PNEUMONIA_dir = '/content/pneumonia_dataset/chest_xray/train/PNEUMONIA'

# Directory with our training NORMAL pictures
train_NORMAL_dir = '/content/pneumonia_dataset/chest_xray/train/NORMAL'

train_PNEUMONIA_names = os.listdir(train_PNEUMONIA_dir)
print(train_PNEUMONIA_names[:10])

train_NORMAL_names = os.listdir(train_NORMAL_dir)
print(train_NORMAL_names[:10])


In [ ]:
print('total training PNEUMONIA images:', len(os.listdir(train_PNEUMONIA_dir)))
print('total training NORMAL images:', len(os.listdir(train_NORMAL_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_PNEUMONIA_pix = [os.path.join(train_PNEUMONIA_dir, fname)
                for fname in train_PNEUMONIA_names[pic_index-8:pic_index]]
next_NORMAL_pix = [os.path.join(train_NORMAL_dir, fname)
                for fname in train_NORMAL_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_PNEUMONIA_pix+next_NORMAL_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up the directories
train_dir = '/content/pneumonia_dataset/chest_xray/train'
test_dir = '/content/pneumonia_dataset/chest_xray/test'
validation_dir = '/content/pneumonia_dataset/chest_xray/val'

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=20,
    class_mode='binary'
)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=20,
    class_mode='binary'
)


In [ ]:

# Model architecture
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

In [ ]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.94:
            print("\nReached 94% accuracy. Stopping training.")
            self.model.stop_training = True

In [ ]:
callback = Callback()
# Model compilation
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=8,
    callbacks=[callback]
)

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array


uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = load_img(path, target_size=(300, 300))
    x = img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=32)

    print("Prediction:", classes[0])
    print("Threshold:", 0.5)

    if classes[0] > 0.5:
        print(fn + " HAS PNEUMONIA")
    else:
        print(fn + " IS NORMAL")
